# 4. Deployment - Local, using Flask

In [177]:
from flask import Flask, request, jsonify, render_template
import pickle
import pandas as pd
import re
from datetime import datetime 
import pandas as pd
from IPython.display import display

# Set the maximum number of rows to display
pd.set_option('display.max_rows', None)



## 4.1 Defining Recommending Engine

In [178]:
# Load dataframes needed
rules_product_types = pd.read_csv('/Users/elizaclapasmac/Desktop/Repositories/Final Project/Final-Project---LHL/Data/3. Recommendation Engines/Rules Product Types.csv')
rules_products = pd.read_csv('/Users/elizaclapasmac/Desktop/Repositories/Final Project/Final-Project---LHL/Data/3. Recommendation Engines/Rules Products.csv')
df_with_counts = pd.read_csv('/Users/elizaclapasmac/Desktop/Repositories/Final Project/Final-Project---LHL/Data/3. Recommendation Engines/Transactions Details.csv')

# # Convert antecedents and consequents to frozensets containing entire product names
# rules_products['antecedents'] = rules_products['antecedents'].apply(lambda x: frozenset([x]))
# rules_products['consequents'] = rules_products['consequents'].apply(lambda x: frozenset([x]))

# Convert antecedents and consequents to frozensets
rules_product_types['antecedents'] = rules_product_types['antecedents'].apply(lambda x: frozenset(eval(x)) if isinstance(x, str) else x)
rules_product_types['consequents'] = rules_product_types['consequents'].apply(lambda x: frozenset(eval(x)) if isinstance(x, str) else x)

In [179]:
import re

def extract_frozenset_contents(frozenset_string):
    """Extract contents from a nested frozenset string and rebuild a flat frozenset."""
    try:
        # Use regex to extract all items within single quotes
        items = re.findall(r"\{'(.*?)'\}", frozenset_string)
        # Return a frozenset of these items (this avoids returning tuples)
        return frozenset(items)
    except Exception as e:
        print(f"Error processing: {frozenset_string} -> {e}")
        return frozenset()

# Apply the extraction function to antecedents and consequents
rules_products['antecedents'] = rules_products['antecedents'].apply(extract_frozenset_contents)
rules_products['consequents'] = rules_products['consequents'].apply(extract_frozenset_contents)

# Check the results
rules_products[['antecedents', 'consequents']].head()


,antecedents,consequents
0,(crudit platter),(fruit platter)
1,(fruit platter),(crudit platter)
2,(crudit platter),(sandwich platter)
3,(sandwich platter),(crudit platter)
4,(scottish shortbread),(assorted holiday cookies)


In [180]:
def recommend_products(input_products, rules_products_df, rules_product_types_df, transactions_df, top_n=5, popular_n=3):
    recommendations = []
    
    # Preprocess input products
    preprocessed_input_products = [preprocess_product_name(product) for product in input_products]
    
    # Convert preprocessed input_products to a frozenset to match the format in the rules dataframe
    input_frozenset = frozenset(preprocessed_input_products)
    
    # Step 1: Try to find product-level associations
    matching_rules = rules_products_df[rules_products_df['antecedents'] == input_frozenset]
    
    if matching_rules.empty:
        # Step 2: If no product-level associations, fallback to product type associations
        
        # Extract the product type of the input products
        input_product_types = transactions_df[transactions_df['Product'].apply(preprocess_product_name).isin(preprocessed_input_products)]['product_type'].unique()
        
        if len(input_product_types) == 0:
            return recommendations  # No valid product types found
        
        # Convert product types to frozensets
        input_product_types_frozenset = frozenset(input_product_types)
        
        # Find matching product type-level rules
        matching_type_rules = rules_product_types_df[rules_product_types_df['antecedents'] == input_product_types_frozenset]
        
        if not matching_type_rules.empty:
            # Sort the matching product type rules by lift, confidence, or another metric to get the top N recommendations
            sorted_type_rules = matching_type_rules.sort_values(by='lift', ascending=False).head(top_n)
            
            # Extract the recommended product types (consequents) from the sorted rules
            for _, rule in sorted_type_rules.iterrows():
                recommendations.append((rule['consequents'], rule['lift'], rule['confidence']))
            
            # Step 3: Recommend popular products within the same product type bought around the same date
            for consequent in sorted_type_rules['consequents']:
                popular_products_within_type = recommend_popular_products_within_type(consequent, transactions_df, popular_n)
                recommendations.extend(popular_products_within_type)
    
        else:
            # If no matching product type-level rules, fallback to just recommending the most popular products
            recommendations.extend(recommend_popular_products(transactions_df, top_n))
    
    else:
        # Sort the matching product-level rules by lift (and optionally other metrics)
        sorted_rules = matching_rules.sort_values(by='lift', ascending=False).head(top_n)
        
        # Extract the recommended products (consequents) from the sorted rules
        for _, rule in sorted_rules.iterrows():
            recommendations.append((rule['consequents'], rule['lift'], rule['confidence']))
    
    return recommendations


In [181]:
# Example usage
input_products = ['apple pie']  # Input from the user or current shopping cart
recommendations = recommend_products(input_products, rules_products, rules_product_types, df_with_counts, top_n=5, popular_n=3)

# Print only the product names or product types, capitalized
for rec in recommendations:
    product_names = [product.title() for product in rec[0]]  # Extract product names and capitalize them
    print(", ".join(product_names))  # Join multiple products if needed


9Inch Pumpkin Pie
Blueberry Pie
12 Dinner Rolls
Raspberry Pie


In [182]:
from flask import Flask, render_template, request, jsonify
import os

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/recommend', methods=['POST'])
def recommend():
    input_products = request.form['products'].split(',')
    input_products = [product.strip() for product in input_products]
    
    # Call the recommendation function
    recommendations = recommend_products(input_products, rules_products, rules_product_types, df_with_counts, top_n=5, popular_n=3)
    
    # Simplify the recommendations to just product names or product types, capitalized
    simplified_recommendations = []
    for rec in recommendations:
        product_names = [product.title() for product in rec[0]]  # Capitalize first letter of each word
        simplified_recommendations.extend(product_names)  # Add to the list

    return render_template('recommend.html', recommendations=simplified_recommendations)

if __name__ == '__main__':
    app.run(debug=True, port=5004, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5004
Press CTRL+C to quit
127.0.0.1 - - [15/Aug/2024 00:26:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2024 00:26:48] "GET /static/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [15/Aug/2024 00:26:48] "GET /static/styles.css HTTP/1.1" 304 -
